In [3]:
import sys
sys.path.append('/workspaces/asclepius/src/Api')

print(sys.path)

['/workspaces/asclepius/src/Api/notebooks', '/usr/local/lib/python311.zip', '/usr/local/lib/python3.11', '/usr/local/lib/python3.11/lib-dynload', '', '/home/vscode/.local/lib/python3.11/site-packages', '/usr/local/lib/python3.11/site-packages', '/workspaces/asclepius/src/Api', '/workspaces/asclepius/src/Api']


In [16]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.kernel import Kernel, KernelArguments
from semantic_kernel.utils.settings import (
    azure_openai_settings_from_dot_env,
)
import pandas as pd
import tiktoken

import Plugins.parse_notes_plugin as pnp

In [28]:
enc = tiktoken.get_encoding('cl100k_base')

In [6]:
note_df = pd.read_csv('/workspaces/asclepius/src/Api/data/notes_no_PHI.csv')

# calculate length of 'NOTE_TEXT' column and add value as another column
note_df['NOTE_TEXT_TOKENS'] = note_df['NOTE_TEXT'].apply(lambda x: len(enc.encode(x)))

note_df['NOTE_TEXT_TOKENS']

0    1560
1    4203
Name: NOTE_TEXT_TOKENS, dtype: int64

In [23]:
plugin = pnp.ParseNotesPlugin()

note_df['CHIEF_COMPLAINT'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_chief_complaint(input=x))
note_df['CHIEF_COMPLAINT_TOKENS'] = note_df['CHIEF_COMPLAINT'].apply(lambda x: len(enc.encode(x)))

note_df['ROS'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_ros(input=x))
note_df['ROS_TOKENS'] = note_df['ROS'].apply(lambda x: len(enc.encode(x)))

note_df['ASSESSMENT'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_assessment(input=x))
note_df['ASSESSMENT_TOKENS'] = note_df['ASSESSMENT'].apply(lambda x: len(enc.encode(x)))

note_df[['NOTE_TEXT_TOKENS', 'CHIEF_COMPLAINT_TOKENS', 'ROS_TOKENS', 'ASSESSMENT_TOKENS']]

#print(note_df['CHIEF_COMPLAINT'][1])


,NOTE_TEXT_TOKENS,CHIEF_COMPLAINT_TOKENS,ROS_TOKENS,ASSESSMENT_TOKENS
0,1560,481,0,447
1,4203,672,100,2624


In [ ]:
print(note_df['ASSESSMENT'][1])

In [40]:
deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
print(endpoint)

kernel = Kernel()
service_id = "default"

kernel.add_service(AzureChatCompletion(
    service_id=service_id,    
    deployment_name=deployment,
    endpoint=endpoint,
    api_key=api_key,
))

plugin = kernel.import_plugin_from_prompt_directory('/workspaces/asclepius/src/Api/Plugins', 'summarize_note')

functions = [
    plugin["summarize_chief_complaint"], 
    plugin["summarize_lab_history"],
    plugin["summarize_assessment"]
]

for idx, note in note_df.iterrows():
    arguments = KernelArguments(input=note["NOTE_TEXT"])
    response = await kernel.invoke(functions, arguments)

    note_df.loc[idx, 'SUM_CHIEF_COMPLAINT'] = response[0].value[0].content
    note_df.loc[idx, 'SUM_LAB_HISTORY'] = response[1].value[0].content
    note_df.loc[idx, 'SUM_ASSESSMENT'] = response[2].value[0].content
    
    note_df.loc[idx, 'SUM_CHIEF_COMPLAINT_TOKENS'] = int(len(enc.encode(response[0].value[0].content)))
    note_df.loc[idx, 'SUM_LAB_HISTORY_TOKENS'] = int(len(enc.encode(response[1].value[0].content)))
    note_df.loc[idx, 'SUM_ASSESSMENT_TOKENS'] = int(len(enc.encode(response[2].value[0].content)))

cols = ['NOTE_TEXT_TOKENS', 'SUM_CHIEF_COMPLAINT_TOKENS', 'SUM_LAB_HISTORY_TOKENS', 'SUM_ASSESSMENT_TOKENS']
note_df[cols]

#chief_complaint = response[0].value[0].content


https://phjirsa-aoai.openai.azure.com/


,NOTE_TEXT_TOKENS,SUM_CHIEF_COMPLAINT_TOKENS,SUM_LAB_HISTORY_TOKENS,SUM_ASSESSMENT_TOKENS
0,1560,213.0,256.0,204.0
1,4203,256.0,256.0,256.0


In [41]:
note_df[cols].dtypes

NOTE_TEXT_TOKENS                int64
SUM_CHIEF_COMPLAINT_TOKENS    float64
SUM_LAB_HISTORY_TOKENS        float64
SUM_ASSESSMENT_TOKENS         float64
dtype: object